In [3]:
import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup,NavigableString, Comment

In [159]:
import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup,NavigableString, Comment
def convert_string(input_str):
    # Regular expression pattern to match a date in the format dd-mm-yyyy
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'
    
    # Search for the pattern in the string
    match = re.search(pattern, input_str)
    if match:
    # Extract and print the date if found
        date_str = match.group(0)
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        date =  date_obj.strftime('%Y-%m-%d')

        return date
    else:
        'No date found'    
#convert time string to datetime object
def convert_time_string(posted_date):
    # Regular expression pattern to match a date in the format dd-mm-yyyy
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'

    # Search for the pattern in the string and extract the date
    match = re.search(pattern, posted_date)
    if match:
        date_str = match.group(0)
        # Convert the extracted date string to datetime object
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        return date_obj
    else:
        return None 
def get_time_string(date_str):
    if len(date_str)<=5:
        crawl_time = datetime.fromtimestamp(time.time())
        year = str(crawl_time.year)
        date_str = date_str+"/"+year
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        date =  date_obj.strftime('%Y-%m-%d')
    else:
        date = convert_string(date_str)
        date_obj = datetime.strptime(date,'%Y-%m-%d')
    return date_obj, date


def remove_div(article):
    divs = article.find_all('div')
    empty_divs = [div for div in divs if not div.text.strip() and not div.contents]
    if not empty_divs:
        return  # No more empty divs, stop recursion
    for div in empty_divs:
        div.decompose()
def get_content_eva(url):
    # def get_content_autodaily()
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('h1', id = 'tieu_de_bai_viet').text.strip()
    date = soup.find('div', class_ = 'eva-author-time-art mar-b-25').text.strip()
    published_date = convert_string(date)
    article = soup.find('article', class_ = 'eva-cont-art no-copy')
    for script_or_style in article(['script', 'style','iframe']):
        script_or_style.decompose()
    element = article.find('div', recursive= True)  # Example: remove comments from the first <div> element
    for comment in element.find_all(string=lambda text: isinstance(text, Comment)):
        comment.decompose()
    related_article =  article.find('div', class_ = 'evtBox evtBoxPrBt')
    related_article.decompose()
    videos = article.find_all('div', align="center")
    for video in videos:
        video.find_next_sibling('p', class_ = 'img_chu_thich_0407 eva-cont-art__cap-img text-center').decompose()
        video.decompose()
    img_container = article.find_all('p', align = "center")
    for container in img_container:
        try:
            caption = container.find_next_sibling('p', class_ ='img_chu_thich_0407 eva-cont-art__cap-img text-center')
            print(caption.text.strip())
            src = container.find('img')['data-original']
            img_tag = soup.new_tag('img')
            img_tag['src'] = src
            caption.insert_before(img_tag)
            figure_tag = soup.new_tag("figure")
            img_tag.wrap(figure_tag)
            
            # Create a new <figcaption> tag and add it after the <img> tag
            figcaption_tag = soup.new_tag("figcaption")
            figcaption_tag.string = caption.text.strip()
            img_tag.insert_after(figcaption_tag)
            figcaption_tag['align'] = "center"
            caption.decompose()
            container.decompose()
        except (TypeError, AttributeError) as e:
            print(e)
    for i in article.find_all('img'):
            i['class'] = "aligncenter"
    source_tag = soup.new_tag('i') 
    source_tag.string = "Nguồn: eva.vn"  # Set the content of <i> tag
    article.append(source_tag)
    for i in article.find_all('div', style="display: none"):
        i.decompose()
    a_tags = article.find_all('a')
    for a_tag in a_tags:
        if len(a_tag.contents) == 1 and a_tag.find('img'):
            a_tag['href'] = 'javascript:void(0)'
    else:
        tag_text = a_tag.get_text()
        # Replace the tag with its text content
        a_tag.replace_with(tag_text)
        a_tag.text.strip()
    h2 =  soup.find_all('h2')[0]
    new_h2 = soup.new_tag("h2")
    new_h2.string = h2.text.strip()
    article.insert(0,h2)
    for i in article.find_all(recursive = True):
        if i.name != 'figcaption':
            try:
                del i['onclick']
                del i['id']
                del i['class']
                del i['style']
            except AttributeError:
                continue
            except TypeError:
                continue
    tags_to_remove = article.find_all(['span'])
    for tag in tags_to_remove:
        # Extract the text from the tag
        tag_text = tag.get_text()
        # Replace the tag with its text content
        tag.replace_with(tag_text)
        tag.text.strip()
    for script_or_style in article(['script', 'style']):
        script_or_style.decompose()
    return article, title, published_date
def get_post(url):
    try:
        content,title,published_date = get_content_eva(url) 
        return content,title,published_date
    except AttributeError as e:
        print(e)
    
def get_list_url(cate_url):
    response = requests.get(cate_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    list = soup.find_all('article', class_ = 'eva-news-trend-h-items d-flex')
    urls = []
    for i in list:
        url = i.find('a')['href']
        urls.append(url)
    return urls
def filter_list(urls):
    filtered_urls = []
    crawl_time = datetime.fromtimestamp(time.time()-2*24*3600)
    for i in range(0,len(urls)):
        try:
            published_date = get_content_eva(urls[i])[2]
            date_posted_norm = datetime.strptime(published_date, '%Y-%m-%d')
            if ( (date_posted_norm.day == crawl_time.day) and (date_posted_norm.month == crawl_time.month) and (date_posted_norm.year == crawl_time.year) ):
                filtered_urls.append(urls[i])
                #print(i)
        except AttributeError as e:
            print(e)
            continue
    return filtered_urls
#add list url to json
#add list url to json
def add_list(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):  
            urls = get_list_url(web_json_obj['urls'][i]['sub-category'][j]['url'])
            print(i,j,web_json_obj['urls'][i]['sub-category'][j]['url'])
            web_json_obj['urls'][i]['sub-category'][j]['url_list'] = filter_list(urls)
# add post content from get content function to json object
def add_post(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):
            web_json_obj['urls'][i]['sub-category'][j]['content'] = {}
            list_key = [v for v in range(0,len(web_json_obj['urls'][i]['sub-category'][j]['url_list']))]
            for u in list_key:
                web_json_obj['urls'][i]['sub-category'][j]['content'][u] = {}
                if u != "":
                    web_json_obj['urls'][i]['sub-category'][j]['content'][u]['text'] ,web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['published_date'] = get_post(web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
                    print(i,j,web_json_obj['urls'][i]['sub-category'][j]['cate_id'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
#add all necessary information to json object
def get_news_eva():
    _eva= {
            "home_page":"https://eva.vn/",
            "urls":{
                "Nhân vật đẹp":
                {
                 "url":"https://eva.vn/nhan-vat-dep-c262.html#",
                 "sub-category":{  
                    0:{"name":"Người đẹp",
                     "url":"https://eva.vn/nhan-vat-dep-c262.html",
                     "cate_id":63,
                      "url_list" : []},
                 }
                }
            }
        }
#
    add_list(_eva)
    add_post(_eva)
    return _eva
#send post content to wordpress via endpoint
def send_post_to_5goals(title,content,category_id,published_date):
    # URL of the API endpoint (this is a placeholder and needs to be replaced with the actual URL)
    url = "https://api2023.5goal.com/wp-json/custom/createPost"
    
    # Data to be sent in the POST request
    data = {
        "title": title,
        "content": content,
        "category_id": category_id,
        "token": 'draftpost',#'5goalvodichcmnl',  # Replace with your actual access token
        "published_date": published_date,
        "domain":"eva"
          # Replace with the actual category ID as required
    }
    
    # Sending the POST request
    response = requests.post(url, data=data)
    
    # Checking the response
    if response.status_code == 200:
        print("The post was successfully created.")
        print("Response:", response.text)  # Prints the response text from the server
    else:
        print(f"Failed to create the post. Status code: {response.status_code}")
def main():
    _eva = get_news_eva()
    for i in list(_eva['urls'].keys()):
    #web_24h_com_vn2['url'][i]['cate_id']
        for j in list(_eva['urls'][i]['sub-category']):
            url_list =  _eva['urls'][i]['sub-category'][j]['url_list']
            print(url_list)
            for t in range(0,len(url_list)):
                content = _eva['urls'][i]['sub-category'][j]['content'][t]['text']
                title = _eva['urls'][i]['sub-category'][j]['content'][t]['title']
                published_date = _eva['urls'][i]['sub-category'][j]['content'][t]['published_date']
                cate_id = _eva['urls'][i]['sub-category'][j]['cate_id']
                print(title, url_list[t])
                #send_post_to_5goals(title,str(content), cate_id, published_date)
                try:
                    text_len = len(content.text)
                    if text_len <450:
                        print(content.text)
                        continue
                    else:
                         #send_post_to_5goals(title,str(content),cate_id,published_date)
                        print("Good job, well done!!!")
                except (AttributeError,TypeError):
                    continue


In [160]:
#test = get_list_url('https://eva.vn/nhan-vat-dep-c262.html')

In [161]:
article = get_content_eva('https://eva.vn/nhan-vat-dep/soi-bua-an-cua-hoa-hau-viet-chua-co-nguoi-ke-nhiem-u55-van-kham-kho-giu-dang-co-nay-c262a578799.html')[0]

Giáng My luôn được khen ngợi có vẻ đẹp vượt thời gian.
Không cần trang điểm, Giáng My vẫn trông đầy trẻ trung nhờ làn da mịn màng.
Cô thường đi bơi lội, vận động khi đi du lịch.
Chạy bộ mỗi buổi sáng là thói quen giúp cơ thể của Hoa hậu Đền Hùng khoẻ khoắn, tràn đầy năng lượng.
Cận cảnh nhan sắc không tì vết của nàng hậu Giáng My ở tuổi ngũ tuần.
Giáng My cung cấp dinh dưỡng bằng yến chưng táo tàu.
Trái cây không thể thiếu trong mỗi bữa ăn của nàng hậu.
'NoneType' object has no attribute 'text'
Bữa tối, cô chỉ ăn một quả bơ của nàng hậu cùng rau củ luộc.
Dù giàu có nhưng nàng hậu ăn uống có vẻ "khiêm tốn", không nạp vào quá nhiều các loại thực phẩm cao lương mĩ vị.
Mỗi buổi sáng, nước trà xanh hay matcha chính là sự lựa chọn ưu tiên của nàng hậu để giúp tăng cường khả năng chống oxy hoá của cơ thể.


In [162]:
article

<article class="eva-cont-art no-copy"><h2>Hoa hậu Giáng My, ăn uống đều đồ ở tuổi U60 giúp nhan sắc bừng sáng mỗi ngày.</h2>
<div>
<div>

</div>
</div>
<!-- Bai Audio -->
<section><div>
<div></div>
<div>

</div>
</div></section> <!-- End Bai Audio -->
<div>
<p>Hoa hậu Giáng My sinh năm 1971, đăng quang cuộc thi Hoa hậu Đền Hùng năm 1992. Từ đó đến nay, cô vẫn được gọi vui là "hoa hậu độc nhất vô nhị" hay "hoa hậu chưa có người kế vị" vì cuộc thi chỉ tổ chức một lần duy nhất. Ở tuổi U60 nàng hậu vẫn lẻ bóng một mình dù đã là mẹ một con, vẻ đẹp và sự giàu có của cô luôn nhận được sự nhiều sự quan tâm của công chúng. </p>
<p>Giáng My luôn làm nhiều đàn em trong showbiz nói chung và người hâm mộ nói riêng phải ngưỡng mộ và trầm trồ vì vẻ đẹp vượt thời gian. Bà mẹ một con có vóc dáng săn chắc, làn da trắng mịn màng như em bé và một mái tóc dài đen nhánh mượt mà. Thập niên 90, Giáng My hoạt động trong vai trò diễn viên, người mẫu, trở thành một trong những "nữ hoàng ảnh lịch" nổi tiếng hàng 

In [122]:
article

<article class="eva-cont-art no-copy"><h2>Nhan sắc của Zia càng lớn càng khiến netizen phải trầm trồ. Nhóc tì như thể phiên bản "tiểu" Marian Rivera song xét khoản tài năng thì cô bé "ăn đứt" mẹ.</h2>
<div>
<div>

</div>
</div>
<!-- Bai Audio -->
<section><div>
<div></div>
<div>

</div>
</div></section> <!-- End Bai Audio -->
<div>
<p>Nói không quá khi cho rằng gia đình Marian Rivera và Dingdong Dantes là tổ ấm quyền lực bậc nhất châu Á. Chỉ tính riêng trang fanpage của Marian Rivera đã có hơn 29 triệu người theo dõi, trang instagram có 12,7 triệu người bấm follow. Gia đình quyền lực của xứ sở biển đảo còn nổi tiếng rần rần trong hội bỉm sữa khi có 2 con: 1 nếp và 1 tẻ đẹp như tranh vẽ.</p>
<p>Trong đó, bé Zia là con gái đầu lòng của cặp đôi quyền lực nhất làng giải trí Philippines. Ngay từ khi mới chào đời, Zia đã khiến cộng đồng mạng châu Á chao đảo bởi thừa hưởng trọn vẹn nhan sắc của mẹ, người được mệnh danh là "Mỹ nhân đẹp nhất Philippines".</p>

<div><section><div>
<div></div>
<d

In [234]:
filter_list(test)

'NoneType' object has no attribute 'insert'
'NoneType' object has no attribute 'insert'
'NoneType' object has no attribute 'insert'
'NoneType' object has no attribute 'insert'
'NoneType' object has no attribute 'insert'
'NoneType' object has no attribute 'insert'


['https://eva.vn/nhan-vat-dep/nhan-sac-a-hau-viet-duy-nhat-duoc-phong-danh-hieu-nghe-si-nhan-dan-tuoi-52-van-la-dai-my-nhan-c262a579272.html',
 'https://eva.vn/nhan-vat-dep/bat-gap-vo-giam-doc-cua-luong-xuan-truong-da-trang-sang-visual-uy-tin-danh-bai-cam-thuong-c262a579195.html',
 'https://eva.vn/nhan-vat-dep/lo-lem-nha-quyen-linh-dien-ao-nhat-binh-hoa-my-nhan-xua-anh-chup-len-lo-nhan-sac-gay-ngo-ngang-c262a579183.html']

In [ ]:
get_news_eva()

Nhân vật đẹp 0 https://eva.vn/nhan-vat-dep-c262.html
https://image-us.eva.vn/upload/4-2023/images/2023-12-06/lo-lem-nha-quyen-linh-17-tuoi-khuon-trang-day-dan-den-phat-sang-dien-ao-dai-nhat-binh-kin-dao-van-ga-408461872_2529124520605479_7543863650240612530_n-1701822463-603-width780height1170.jpg
https://image-us.eva.vn/upload/4-2023/images/2023-12-06/lo-lem-nha-quyen-linh-17-tuoi-khuon-trang-day-dan-den-phat-sang-dien-ao-dai-nhat-binh-kin-dao-van-ga-408461872_2529124520605479_7543863650240612530_n-1701822463-603-width780height1170.jpg
https://image-us.eva.vn/upload/4-2023/images/2023-12-06/lo-lem-nha-quyen-linh-17-tuoi-khuon-trang-day-dan-den-phat-sang-dien-ao-dai-nhat-binh-kin-dao-van-ga-408461872_2529124520605479_7543863650240612530_n-1701822463-603-width780height1170.jpg
https://image-us.eva.vn/upload/4-2023/images/2023-12-06/lo-lem-nha-quyen-linh-17-tuoi-khuon-trang-day-dan-den-phat-sang-dien-ao-dai-nhat-binh-kin-dao-van-ga-408461872_2529124520605479_7543863650240612530_n-170182246

https://image-us.eva.vn/upload/4-2023/images/2023-12-06/lo-lem-nha-quyen-linh-17-tuoi-khuon-trang-day-dan-den-phat-sang-dien-ao-dai-nhat-binh-kin-dao-van-ga-408461872_2529124520605479_7543863650240612530_n-1701822463-603-width780height1170.jpg


<article class="eva-cont-art no-copy"><h2>Lọ Lem nhà Quyền Linh khoe vẻ đẹp "khuôn trăng đầy đặn" trong bộ ảnh mới. Mới 17 tuổi đã trổ mã làm "đại mỹ nhân".</h2>
<div>
<div>

</div>
</div>
<!-- Bai Audio -->
<section><div>
<div></div>
<div>

</div>
</div></section> <!-- End Bai Audio -->
<div>
<p>Trong dàn nhóc tì Vbiz, Lọ Lem nhà Quyền Linh nổi bật nhất nhì khi được dự đoán là Hoa hậu tương lai. Cô bé sở hữu vẻ đẹp thanh thoát và sang trọng hệt tiên tử. Mới đây, con gái MC nổi tiếng vừa thực hiện một bộ ảnh lấy cảm hứng từ phụ nữ xưa khi khoác trên mình mẫu áo dài Nhật Bình. Nhờ gương mặt không khuyết điểm cũng như chiều cao ấn tượng mà Lọ Lem chiếm trọn tình cảm của người hâm mộ. </p>
<div><section><div>
<div></div>
<div>

</div>
</div></section></div>
[caption id="" align="aligncenter" width="800"]<img src="https://image-us.eva.vn/upload/4-2023/images/2023-12-06/lo-lem-nha-quyen-linh-17-tuoi-khuon-trang-day-dan-den-phat-sang-dien-ao-dai-nhat-binh-kin-dao-van-ga-408461872_25291245206

In [236]:
main()

Nhân vật đẹp 0 https://eva.vn/nhan-vat-dep-c262.html
'NoneType' object has no attribute 'insert'
'NoneType' object has no attribute 'insert'
'NoneType' object has no attribute 'insert'
'NoneType' object has no attribute 'insert'
'NoneType' object has no attribute 'insert'
'NoneType' object has no attribute 'insert'
Nhân vật đẹp 0 63 Người đẹp Người đẹp Nhan sắc Á hậu Việt duy nhất được phong danh hiệu Nghệ sĩ nhân dân, tuổi 52 vẫn là đại mỹ nhân https://eva.vn/nhan-vat-dep/nhan-sac-a-hau-viet-duy-nhat-duoc-phong-danh-hieu-nghe-si-nhan-dan-tuoi-52-van-la-dai-my-nhan-c262a579272.html
Nhân vật đẹp 0 63 Người đẹp Người đẹp Bắt gặp vợ giám đốc của Lương Xuân Trường: Da trắng sáng, visual uy tín "đánh bại" cam thường https://eva.vn/nhan-vat-dep/bat-gap-vo-giam-doc-cua-luong-xuan-truong-da-trang-sang-visual-uy-tin-danh-bai-cam-thuong-c262a579195.html
Nhân vật đẹp 0 63 Người đẹp Người đẹp Lọ Lem nhà Quyền Linh diện áo Nhật Bình hóa mỹ nhân xưa, ảnh chụp lén lộ nhan sắc gây ngỡ ngàng https://ev

In [200]:
test = get_list_url('https://eva.vn/nhan-vat-dep-c262.html')

'https://eva.vn/nhan-vat-dep/co-giao-yoga-dep-nhat-chau-a-noi-tieng-10-nam-truoc-gio-ra-sao-duong-cong-van-chuan-than-ve-nu-c262a579220.html'

In [185]:
get_post('https://eva.vn/nhan-vat-dep/nhan-sac-a-hau-viet-duy-nhat-duoc-phong-danh-hieu-nghe-si-nhan-dan-tuoi-52-van-la-dai-my-nhan-c262a579272.html')

(<article class="eva-cont-art no-copy"><h2 class="" id="" style="">Trịnh Kim Chi được xem là bông hoa tài sắc vẹn toàn, làm rạng danh làng sắc đẹp Việt.</h2>
 <div class="" id="" style="">
 <div class="" id="" style="">
 
 </div>
 </div>
 <!-- Bai Audio -->
 <section class="" id="" style=""><div class="" id="" style="">
 <div class="" id="" style=""><span class="" id="" style=""></span></div>
 <div class="" id="" style="">
 
 </div>
 </div></section> <!-- End Bai Audio -->
 <div class="" id="" style="">
 <p class="" id="" style="">Trong 42 cái tên nghệ sĩ mới đây được phong tặng danh hiệu “Nghệ sĩ nhân dân” (NSND) thuộc nhiều lĩnh vực như Âm nhạc, Điện ảnh, Múa, Phát thanh - Truyền hình và Sân khấu..., Trịnh Kim Chi là một trong những mỹ nhân được chú ý nhiều nhất. Đó là bởi cô được xem là Á hậu đầu tiên được phong tặng danh hiệu NSND. Trước đó, cô nhận danh hiệu NSƯT vào năm 2014 và cũng là nàng hậu xuất sắc nhất trong làng sắc đẹp Việt trên màn ảnh. </p>
 
 [caption id="" align="alig

In [92]:
response = requests.get('https://eva.vn/nhan-vat-dep/lo-lem-nha-quyen-linh-dien-ao-nhat-binh-hoa-my-nhan-xua-anh-chup-len-lo-nhan-sac-gay-ngo-ngang-c262a579183.html')
soup = BeautifulSoup(response.content, 'html.parser')
title = soup.find('h1', id = 'tieu_de_bai_viet').text.strip()
date = soup.find('div', class_ = 'eva-author-time-art mar-b-25').text.strip()
published_date = convert_string(date)
article = soup.find('article', class_ = 'eva-cont-art no-copy')
for script_or_style in article(['script', 'style','iframe']):
    script_or_style.decompose()
element = article.find('div', recursive= True)  # Example: remove comments from the first <div> element
for comment in element.find_all(string=lambda text: isinstance(text, Comment)):
    comment.decompose()
related_article =  article.find('div', class_ = 'evtBox evtBoxPrBt')
related_article.decompose()
videos = article.find_all('div', align="center")
for video in videos:
    video.find_next_sibling('p', class_ = 'img_chu_thich_0407 eva-cont-art__cap-img text-center').decompose()
    video.decompose()
img_container = article.find_all('p', align = "center")
for container in img_container:
    try:
        caption = container.find_next_sibling('p', class_ ='img_chu_thich_0407 eva-cont-art__cap-img text-center')
        print(caption.text.strip())
        src = container.find('img')['data-original']
        img_tag = soup.new_tag('img')
        img_tag['src'] = src
        caption.insert_before(img_tag)
        caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
        caption_text = NavigableString(caption.text.strip())
        caption_end = NavigableString("[/caption]")
            # Insert the custom tags and caption text around the <img> tag
        img_tag.insert_before(caption_start)
        img_tag.insert_after(caption_end)
        img_tag.insert_after(caption_text)
        caption.decompose()
        container.decompose()
    except (TypeError, AttributeError) as e:
        print(e)
for i in article.find_all('img'):
        i['class'] = "aligncenter"
source_tag = soup.new_tag('i') 
source_tag.string = "Nguồn: eva.vn"  # Set the content of <i> tag
article.append(source_tag)
for i in article.find_all('div', style="display: none"):
    i.decompose()
for a_tag in article.find_all('a'):
    a_tag['href'] = 'javascript:void(0)'
h2 =  soup.find_all('h2')[0]
new_h2 = soup.new_tag("h2")
new_h2.string = h2.text.strip()
article.insert(0,h2)
"""first_img_container = article.find('div', id = 'container-24h-banner-in-image').parent
first_cap = first_img_container.find_next_sibling('p', class_ ='img_chu_thich_0407 eva-cont-art__cap-img text-center')
first_img = first_img_container.find('img')

img_tag = soup.new_tag('img')
img_tag['src'] = first_img['data-original']
print(img_tag)
print(first_cap)
first_cap.insert_before(img_tag)
caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
caption_text = NavigableString(first_cap.text.strip())
caption_end = NavigableString("[/caption]")
    # Insert the custom tags and caption text around the <img> tag
img_tag.insert_before(caption_start)
img_tag.insert_after(caption_end)
img_tag.insert_after(caption_text)
first_img_container.decompose()
first_cap.decompose()

for caption in article.find_all('p', class_ = "img_chu_thich_0407 eva-cont-art__cap-img text-center")[1:]:
    img_container = caption.find_previous_sibling()
    try:
        src = img_container.find('img')['data-original']
        img_tag = soup.new_tag('img')
        img_tag['src'] = src
        caption.insert_before(img_tag)
        caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
        caption_text = NavigableString(caption.text.strip())
        caption_end = NavigableString("[/caption]")
            # Insert the custom tags and caption text around the <img> tag
        img_tag.insert_before(caption_start)
        img_tag.insert_after(caption_end)
        img_tag.insert_after(caption_text)
        img_container.decompose()
        caption.decompose()
        #print(src)
    except TypeError:
        continue
for i in article.find_all('img'):
        i['class'] = "aligncenter"
source_tag = soup.new_tag('i') 
source_tag.string = "Nguồn: eva.vn"  # Set the content of <i> tag
article.append(source_tag)
for i in article.find_all('div', style="display: none"):
    i.decompose()
for a_tag in article.find_all('a'):
    a_tag['href'] = 'javascript:void(0)'
h2 =  soup.find_all('h2')[0]
new_h2 = soup.new_tag("h2")
new_h2.string = h2.text.strip()
article.insert(0,h2)"""

Con gái Quyền Linh thả dáng trong bộ ảnh mới, chẳng cần hở hang vẫn gây mê vì visual điểm 10.
'NoneType' object has no attribute 'text'
Vẻ đẹp thanh nhã, sang trọng chính là điểm cộng lớn nhất của Lọ Lem. Đặc biệt trong lần xuất hiện này, Lọ Lem còn tô vẽ gương mặt vô cùng kỹ lưỡng để phù hợp với hình tượng xưa.
'NoneType' object has no attribute 'text'
Con gái Quyền Linh sở hữu vẻ đẹp khó ai bì kịp, được dự đoán là Hoa hậu tương lai.
'NoneType' object has no attribute 'text'
Thảo Linh sở hữu ngũ quan hài hòa, nụ cười sáng và đôi mắt biết cười nên dễ dàng thu hút người đối diện ngay từ giây đầu tiên.
'NoneType' object has no attribute 'text'
Nhóc tì cũng sở hữu chiều cao nổi bật, mới 17 tuổi đã cao hơn 1m7.
'NoneType' object has no attribute 'text'
Trong đoạn clip ghi lại cảnh hậu trường, dù hình ảnh chất lượng thấp nhưng visual như mỹ nhân xưa của Lọ Lem vẫn chất lượng cao, nhìn góc nào cũng long lanh.
Lọ Lem biến hóa lạ lẫm vẫn chiếm spotlight vì quá xinh đẹp.
'NoneType' object has n

'first_img_container = article.find(\'div\', id = \'container-24h-banner-in-image\').parent\nfirst_cap = first_img_container.find_next_sibling(\'p\', class_ =\'img_chu_thich_0407 eva-cont-art__cap-img text-center\')\nfirst_img = first_img_container.find(\'img\')\n\nimg_tag = soup.new_tag(\'img\')\nimg_tag[\'src\'] = first_img[\'data-original\']\nprint(img_tag)\nprint(first_cap)\nfirst_cap.insert_before(img_tag)\ncaption_start = NavigableString("[caption id="" align="aligncenter" width="800"]")\ncaption_text = NavigableString(first_cap.text.strip())\ncaption_end = NavigableString("[/caption]")\n    # Insert the custom tags and caption text around the <img> tag\nimg_tag.insert_before(caption_start)\nimg_tag.insert_after(caption_end)\nimg_tag.insert_after(caption_text)\nfirst_img_container.decompose()\nfirst_cap.decompose()\n\nfor caption in article.find_all(\'p\', class_ = "img_chu_thich_0407 eva-cont-art__cap-img text-center")[1:]:\n    img_container = caption.find_previous_sibling()\n 

In [93]:
article

<article class="eva-cont-art no-copy"><h2 class="fw-semi-bold tuht_show">Lọ Lem nhà Quyền Linh khoe vẻ đẹp "khuôn trăng đầy đặn" trong bộ ảnh mới. Mới 17 tuổi đã trổ mã làm "đại mỹ nhân".</h2>
<div class="eva-cont-art__sum">
<div class="nsSap sap clrF">

</div>
</div>
<!-- Bai Audio -->
<section class="banr-hot mrT20 mrB20" id="bnBAER_119_15s_container"><div class="banrpstn">
<div class="txtbanrHrzt txtrt" id="bnbaer_by_bnBAER_119_15s"><span class="displaynone"></span></div>
<div class="" id="bnBAER_119_15s">

</div>
</div></section> <!-- End Bai Audio -->
<div class="eva-cont-art__info" id="baiviet-container">
<p>Trong dàn nhóc tì Vbiz, Lọ Lem nhà Quyền Linh nổi bật nhất nhì khi được dự đoán là Hoa hậu tương lai. Cô bé sở hữu vẻ đẹp thanh thoát và sang trọng hệt tiên tử. Mới đây, con gái MC nổi tiếng vừa thực hiện một bộ ảnh lấy cảm hứng từ phụ nữ xưa khi khoác trên mình mẫu áo dài Nhật Bình. Nhờ gương mặt không khuyết điểm cũng như chiều cao ấn tượng mà Lọ Lem chiếm trọn tình cảm của

In [71]:
article

<article class="eva-cont-art no-copy"><h2 class="fw-semi-bold tuht_show">Nhan sắc của Zia càng lớn càng khiến netizen phải trầm trồ. Nhóc tì như thể phiên bản "tiểu" Marian Rivera song xét khoản tài năng thì cô bé "ăn đứt" mẹ.</h2>
<div class="eva-cont-art__sum">
<div class="nsSap sap clrF">

</div>
</div>
<!-- Bai Audio -->
<section class="banr-hot mrT20 mrB20" id="bnBAER_119_15s_container"><div class="banrpstn">
<div class="txtbanrHrzt txtrt" id="bnbaer_by_bnBAER_119_15s"><span class="displaynone"></span></div>
<div class="" id="bnBAER_119_15s">

</div>
</div></section> <!-- End Bai Audio -->
<div class="eva-cont-art__info" id="baiviet-container">
<p>Nói không quá khi cho rằng gia đình Marian Rivera và Dingdong Dantes là tổ ấm quyền lực bậc nhất châu Á. Chỉ tính riêng trang fanpage của Marian Rivera đã có hơn 29 triệu người theo dõi, trang instagram có 12,7 triệu người bấm follow. Gia đình quyền lực của xứ sở biển đảo còn nổi tiếng rần rần trong hội bỉm sữa khi có 2 con: 1 nếp và 1 t

In [47]:
article.find_all('p', class_ = 'img_chu_thich_0407 eva-cont-art__cap-img text-center')[1:]

[<p class="img_chu_thich_0407 eva-cont-art__cap-img text-center">Em bé được trang điểm nhẹ nhàng và làm tóc xoăn xõa, đầu đội vương miện.</p>,
 <p class="img_chu_thich_0407 eva-cont-art__cap-img text-center">Tấm hình chụp này còn khiến nhiều fans trầm trồ vì quá giống Marian Rivera. Nhiều comment ví von Zia là tiểu Marian Rivera.</p>,
 <p class="img_chu_thich_0407 eva-cont-art__cap-img text-center">Zia là tinh túy của cả bố lẫn mẹ, song em bé có lẽ giống mẹ hơn cả. Tấm hình chụp ngày bé của Marian Rivera khiến nhiều người bối rối vì cứ ngỡ ảnh của Zia đã được chỉnh màu.</p>,
 <p class="img_chu_thich_0407 eva-cont-art__cap-img text-center"> Càng lớn, Zia càng xinh đẹp và được dự đoán sẽ "tiếp quản" danh hiệu "Mỹ nhân đẹp nhất Philippines" của mẹ trong tương lai. Không chỉ xinh đẹp, dưới sự nuôi dạy của mẹ, Zia còn thông minh, học giỏi.</p>,
 <p class="img_chu_thich_0407 eva-cont-art__cap-img text-center">Zia càng lớn càng hoàn thiện ngũ quan trên gương mặt. Nhóc tì mang vẻ lém lỉnh với 

In [142]:
for caption in article.find_all('p', class_ = "img_chu_thich_0407 eva-cont-art__cap-img text-center"):
    img_container = caption.find_previous_sibling()
    try:
        src = img_container.find('img')['data-original']
        img_tag = soup.new_tag('img')
        img_tag['src'] = src
        caption.insert_before(img_tag)
        caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
        caption_text = NavigableString(caption.text.strip())
        caption_end = NavigableString("[/caption]")
            # Insert the custom tags and caption text around the <img> tag
        img_tag.insert_before(caption_start)
        img_tag.insert_after(caption_end)
        img_tag.insert_after(caption_text)
        img_container.decompose()
        caption.decompose()
        print(src)
    except TypeError:
        continue
for i in article.find_all('img'):
        i['class'] = "aligncenter"
source_tag = soup.new_tag('i') 
source_tag.string = "Nguồn: vtv.vn"  # Set the content of <i> tag
article.append(source_tag)
for i in article.find_all('div', style="display: none"):
    i.decompose()
for a_tag in article.find_all('a'):
    a_tag['href'] = 'javascript:void(0)'


In [252]:
response = requests.get('https://eva.vn/nhan-vat-dep/lo-lem-nha-quyen-linh-dien-ao-nhat-binh-hoa-my-nhan-xua-anh-chup-len-lo-nhan-sac-gay-ngo-ngang-c262a579183.html')
soup = BeautifulSoup(response.content, 'html.parser')
title = soup.find('h1', id = 'tieu_de_bai_viet').text.strip()
date = soup.find('div', class_ = 'eva-author-time-art mar-b-25').text.strip()
published_date = convert_string(date)

#h2_tag = soup.new_tag('h2')
#h2_tag.string = h2
article.insert(0,soup.find('h2'))
article = soup.find('article', class_ = 'eva-cont-art no-copy')
for script_or_style in article(['script', 'style','iframe']):
    script_or_style.decompose()
element = article.find('div', recursive= True)  # Example: remove comments from the first <div> element
for comment in element.find_all(string=lambda text: isinstance(text, Comment)):
    comment.decompose()
related_article =  article.find('div', class_ = 'evtBox evtBoxPrBt')
related_article.decompose()
videos = article.find_all('div', align="center")
for video in videos:
    video.find_next_sibling('p', class_ = 'img_chu_thich_0407 eva-cont-art__cap-img text-center').decompose()
    video.decompose()
first_cap_container = article.find('div', id = 'container-24h-banner-in-image')

In [260]:
print(article.find('div', id = 'container-24h-banner-in-image').find_next_sibling('p', align="center"))

None


In [258]:
print(first_cap_container.find_next_sibling('p'))

None


In [267]:
article.find_all('p', class_ = 'img_chu_thich_0407 eva-cont-art__cap-img text-center')[0].find_next_sibling('p', class_ ='img_chu_thich_0407 eva-cont-art__cap-img text-center')

<p align="center" class="img_chu_thich_0407 eva-cont-art__cap-img text-center">Con gái Quyền Linh thả dáng trong bộ ảnh mới, chẳng cần hở hang vẫn gây mê vì visual điểm 10. </p>

In [263]:
print(article.prettify())

<article class="eva-cont-art no-copy">
 <div class="eva-cont-art__sum">
  <div class="nsSap sap clrF">
  </div>
 </div>
 <!-- Bai Audio -->
 <section class="banr-hot mrT20 mrB20" id="bnBAER_119_15s_container">
  <div class="banrpstn">
   <div class="txtbanrHrzt txtrt" id="bnbaer_by_bnBAER_119_15s">
    <span class="displaynone">
    </span>
   </div>
   <div class="" id="bnBAER_119_15s">
   </div>
  </div>
 </section>
 <!-- End Bai Audio -->
 <div class="eva-cont-art__info" id="baiviet-container">
  <p>
   Trong dàn nhóc tì Vbiz, Lọ Lem nhà Quyền Linh nổi bật nhất nhì khi được dự đoán là Hoa hậu tương lai. Cô bé sở hữu vẻ đẹp thanh thoát và sang trọng hệt tiên tử. Mới đây, con gái MC nổi tiếng vừa thực hiện một bộ ảnh lấy cảm hứng từ phụ nữ xưa khi khoác trên mình mẫu áo dài Nhật Bình. Nhờ gương mặt không khuyết điểm cũng như chiều cao ấn tượng mà Lọ Lem chiếm trọn tình cảm của người hâm mộ.
  </p>
  <p align="center" class="img_chu_thich_0407 eva-cont-art__cap-img text-center">
   <di

In [130]:
for caption in article.find_all('p', class_ = "img_chu_thich_0407 eva-cont-art__cap-img text-center"):
    caption.decompose()

In [137]:
for i in article.find_all('img'):
    i['href'] = 'javascript:void(0)'
    print(i)

<img alt="Con gái mỹ nhân đẹp nhất Philippines lên 8 tuổi, có răng thỏ vẫn là công chúa đời thực - 1" class="news-image" data-original="https://image-us.eva.vn/upload/4-2023/images/2023-12-04/con-gai-rang-tho-cua-marian-rivera-len-8-tu-nhan-sac-cho-toi-tai-nang-an-dut-me-406830486_911137673715071_1644369503908178404_n-1701682752-148-width780height1208.jpg" href="javascript:void(0)" onclick="return openNewImage(this, '')" src="data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs=" width="660">
<div id="24h-banner-in-image" style="position: absolute;display: block;width: 660px;height: 90px;color: white;bottom: 0px;left: 0px; right: 0px;margin: 0 auto">
<div class="txtCent" id="bnBAER_165_15s">
<div class="text_adver_right" onclick="in_image_close();"></div>

</div>
<div id="24h-banner-in-image-close" style="top:5px;right: 5px;position: absolute;display: none;overflow: hidden;width: 22px;height: 22px;color: #ccc;font-size: large;">
<img alt="Con gái mỹ nhân đẹp nhất Philippines lên 8

In [105]:
article.find_all('div', style="display: none")
   

[<div id="tracking_auto_am_thanh_preroll_mutezplayer_aea631" style="display: none"></div>,
 <div id="tracking_auto_am_thanh_preroll_unmutezplayer_aea631" style="display: none"></div>,
 <div id="tracking_user_am_thanh_preroll_mutezplayer_aea631" style="display: none"></div>,
 <div id="tracking_user_am_thanh_preroll_unmutezplayer_aea631" style="display: none"></div>,
 <div id="onplay_tracking_durationzplayer_aea631" style="display: none"></div>,
 <div id="tracking_duration_firstzplayer_aea631" style="display: none"></div>,
 <div id="tracking_duration_secondzplayer_aea631" style="display: none"></div>,
 <div id="tracking_duration_thirdzplayer_aea631" style="display: none"></div>,
 <div id="tracking_duration_completezplayer_aea631" style="display: none"></div>]

In [99]:
list_captions = []
for p_tag in article.find_all('p', class_ = 'img_chu_thich_0407 eva-cont-art__cap-img text-center'):
    if p_tag.text != None:
        list_captions.append(p_tag.text.strip())


In [100]:
list_captions

['Nhóc tì mới đây vừa được tổ chức bữa tiệc sinh nhật 8 tuổi linh đình. Em bé khoe sắc vóc như công chúa trong truyện cổ tích với chiếc đầm xòe xinh xắn.',
 'Con gái gia đình giàu có bậc nhất Philippines lộng lẫy như truyện cổ tích Bạch Tuyết và 7 chú lùn.',
 'Em bé được trang điểm nhẹ nhàng và làm tóc xoăn xõa, đầu đội vương miện.',
 'Tấm hình chụp này còn khiến nhiều fans trầm trồ vì quá giống Marian Rivera. Nhiều comment ví von Zia là tiểu Marian Rivera.',
 'Zia là tinh túy của cả bố lẫn mẹ, song em bé có lẽ giống mẹ hơn cả. Tấm hình chụp ngày bé của Marian Rivera khiến nhiều người bối rối vì cứ ngỡ ảnh của Zia đã được chỉnh màu.',
 'Càng lớn, Zia càng xinh đẹp và được dự đoán sẽ "tiếp quản" danh hiệu "Mỹ nhân đẹp nhất Philippines" của mẹ trong tương lai. Không chỉ xinh đẹp, dưới sự nuôi dạy của mẹ, Zia còn thông minh, học giỏi.',
 'Zia càng lớn càng hoàn thiện ngũ quan trên gương mặt. Nhóc tì mang vẻ lém lỉnh với đôi mắt cười, khuôn miệng tươi tắn và đặc biệt là chiếc răng thỏ.',
 

In [41]:
for i in article.find_all('div', class_  = 'container-24h-banner-in-image'):
    #next_sibling= i.find_next_sibling('p', class_ = 'img_chu_thich_0407 eva-cont-art__cap-img text-center')
   
    print(i)
    
    

In [132]:
article.find_all('a')

[]

In [81]:
html_content = '''
<html>
  <body>
    <div class="header">Header content</div>
    <div class="main">Main content</div>
    <div class="footer">Footer content</div>
  </body>
</html>
'''
soup = BeautifulSoup(html_content, 'html.parser')
current_element = soup.find(class_='footer')
previous_sibling = current_element.find_previous_sibling()
print(previous_sibling)
# To find the previous sibling with a specific class:
# previous_sibling = current_element.find_previous_sibling(class_='specific-class')


<div class="main">Main content</div>
